In [1]:
import dataloader
import numpy as np
import pandas as pd
import plotly.express as px
import geopy
from schemas import Trip
import datetime as dt

In [2]:
trips = dataloader.TripsLoader('03-07-2022')

In [3]:
#Machine numbers present for given day
trips._machines.keys()

dict_keys([20, 13, 25, 11, 46, 37, 52, 15, 19, 22, 26, 5, 44, 16, 30, 36, 23, 1, 38, 24, 41, 8, 4, 50, 49])

In [15]:
machine_nb = 44 #Choice of machine number
machine_nb_info = trips._machines[machine_nb]#[machine for machine in trips._machines if machine.machine_id==machine_nb][0]
all_trips_for_machine = machine_nb_info.trips
[t.quantity for t in all_trips_for_machine]

[27.0, 27.0, 27.0, 27.0, 27.0, 27.0, 27.0, 27.0, 27.0, 27.0, 27.0, 27.0]

In [546]:
trip_durations = []
for k in trips._machines.keys():
    machine_k = trips._machines[k]
    trip_durations.append([trip.duration for trip in machine_k.trips])

trip_durations = [item for sublist in trip_durations for item in sublist]

In [556]:
min(trip_durations)
#max(trip_durations)

0.11666666666666667

In [552]:
fig = px.histogram(trip_durations, nbins=1000)
fig.show()

In [494]:
#Look at actual trip recordings
start_end_each_trip_dict = [dict(Start=trips.start_date, End=trips.end_date) for trips in all_trips_for_machine]
df_pyplot = pd.DataFrame(start_end_each_trip_dict)

fig = px.timeline(df_pyplot, x_start="Start", x_end="End")
fig.update_yaxes(autorange="reversed") # otherwise tasks are listed from the bottom up
fig.show()


In [495]:
# Collect all positions throughout day
# Initialize first data point as load spot of trip 1 => Start trip 1
# Iteratively run next points through an algo that decides if load or dump, and start new trip each time load is decided (either explicitly or only implicitly)

In [496]:
len(all_trips_for_machine)

20

In [497]:
all_positions_and_times = [trip.positions for trip in all_trips_for_machine]

In [498]:
all_positions_and_times =  [item for sublist in all_positions_and_times for item in sublist]

In [499]:
all_positions_and_times

[Position(lat=59.939565835576445, lon=10.420865155766178, uncertainty=5.0, timestamp=Timestamp('2022-03-07 06:07:02.996000+0000', tz='UTC')),
 Position(lat=59.93955711839715, lon=10.420881668115426, uncertainty=5.0, timestamp=Timestamp('2022-03-07 06:11:21.988000+0000', tz='UTC')),
 Position(lat=59.93954462936142, lon=10.420932797724772, uncertainty=5.0, timestamp=Timestamp('2022-03-07 06:11:24.988000+0000', tz='UTC')),
 Position(lat=59.93953289469698, lon=10.420994488532116, uncertainty=5.0, timestamp=Timestamp('2022-03-07 06:11:26.988000+0000', tz='UTC')),
 Position(lat=59.93952748836944, lon=10.42103187182026, uncertainty=5.0, timestamp=Timestamp('2022-03-07 06:11:27.994000+0000', tz='UTC')),
 Position(lat=59.93952254304656, lon=10.421073026964832, uncertainty=5.0, timestamp=Timestamp('2022-03-07 06:11:28.994000+0000', tz='UTC')),
 Position(lat=59.9395181425474, lon=10.421119043613244, uncertainty=5.0, timestamp=Timestamp('2022-03-07 06:11:29.995000+0000', tz='UTC')),
 Position(lat=

In [500]:
all_positions_and_times = sorted(all_positions_and_times, key=lambda Position: Position.timestamp) #Is it actually sorted now

In [501]:
##Algorithm - first Position is first recording of trip 1.
## Have some sort of placeholder for current trip
## Add location for predicted load spot
## Add new positions to the trip
## If predicted dump -> Record dump position
## Add new positions to the trip
## If predicted load -> Stop trip
## Reiterate until EOL all positions
##Should maybe throw error/warning if times is not sorted etc. or locations jumps unreasonably much

##Possible logic for dump and load
## - Minimal movement around predicted time
## - In vicinity of spots of loading and dumping -> should be made available prior to day -> Or decide based on yesterdays spot -> i.e. within 500m from last days spot -> can be prompted by driver if they are going to new location

In [553]:
automated_trips = []
temp_positions = []
trip_counter = 0
predicted_dump = False #Must have predicted dump for possible load prediction
speed_threshold = 0.015

#Initialize day
temp_positions.append(all_positions_and_times[0])

temp_load_pos = all_positions_and_times[0]
#temp_dump_pos = temp_load_pos #Just to initialize


for pos in all_positions_and_times[1:]:
    temp_positions.append(pos)

    #Some logic, e.g. if movement less than 3m/s since last, predict spot
    current_time = pos.timestamp
    current_lat_lon = (pos.lat, pos.lon)
    prev_time = temp_positions[-2].timestamp
    prev_lat_lon = (temp_positions[-2].lat,temp_positions[-2].lon)
    seconds_gone = (current_time.to_pydatetime()-prev_time.to_pydatetime()).total_seconds()
    meters_driven = geopy.distance.geodesic(prev_lat_lon, current_lat_lon).m

    if meters_driven/(seconds_gone+0.1) < speed_threshold and predicted_dump == False:
        #Reached predicted dump spot
        predicted_dump = True
        temp_dump_pos = pos

    if meters_driven/(seconds_gone+0.1) < speed_threshold and predicted_dump == True:
        #Reached predicted load spot, trip is over
        
        temp_trip_id = str(trip_counter) #Placeholder

        automated_trips.append(Trip(trip_id=temp_trip_id, load='Stone', quantity=10.0, positions = temp_positions, dump_latlon=[temp_dump_pos.lat, temp_dump_pos.lon], load_latlon=[temp_load_pos.lat, temp_load_pos.lon]))

        #Reinitialize/increment variables
        temp_load_pos = pos
        predicted_dump = False
        trip_counter += 1
        temp_positions = [pos]





In [554]:
len(automated_trips)

34

In [504]:
automated_trips

[Trip(trip_id='0', load='Stone', quantity=10.0, positions=[Position(lat=59.939565835576445, lon=10.420865155766178, uncertainty=5.0, timestamp=Timestamp('2022-03-07 06:07:02.996000+0000', tz='UTC')), Position(lat=59.93955711839715, lon=10.420881668115426, uncertainty=5.0, timestamp=Timestamp('2022-03-07 06:11:21.988000+0000', tz='UTC'))], dump_latlon=(59.93955711839715, 10.420881668115426), load_latlon=(59.939565835576445, 10.420865155766178), latlons=[(59.939565835576445, 10.420865155766178), (59.93955711839715, 10.420881668115426)], length=0.0013398770429414847, duration=4.316533333333334),
 Trip(trip_id='1', load='Stone', quantity=10.0, positions=[Position(lat=59.93955711839715, lon=10.420881668115426, uncertainty=5.0, timestamp=Timestamp('2022-03-07 06:11:21.988000+0000', tz='UTC')), Position(lat=59.93954462936142, lon=10.420932797724772, uncertainty=5.0, timestamp=Timestamp('2022-03-07 06:11:24.988000+0000', tz='UTC')), Position(lat=59.93953289469698, lon=10.420994488532116, uncer

In [555]:
#Look at predicted trip recordings
start_end_each_trip_dict_automated = [dict(Start=trips.positions[0].timestamp, End=trips.positions[-1].timestamp) for trips in automated_trips]
df_pyplot_automated  = pd.DataFrame(start_end_each_trip_dict_automated)

fig = px.timeline(df_pyplot_automated, x_start="Start", x_end="End")
fig.update_yaxes(autorange="reversed") # otherwise tasks are listed from the bottom up
fig.show()

In [506]:
from plotly.subplots import make_subplots

fig_actual = px.timeline(df_pyplot, x_start="Start", x_end="End")
fig_actual.update_yaxes(autorange="reversed")

fig_automated = px.timeline(df_pyplot_automated, x_start="Start", x_end="End")
fig_automated.update_yaxes(autorange="reversed") # otherwise tasks are listed from the bottom up

fig_sub = make_subplots(rows=2, shared_xaxes=True)
fig_sub.append_trace(fig_actual['data'][0], row=1, col=1)
fig_sub.add_annotation(xref="x domain",yref="y domain",x=0.5, y=1.2, showarrow=False,
                   text="<b>Actual recorded trips</b>", row=1, col=1)
fig_sub.append_trace(fig_automated['data'][0], row=2, col=1)
fig_sub.add_annotation(xref="x domain",yref="y domain",x=0.5, y=1.2, showarrow=False,
                   text="<b>Automated recorded trips</b>", row=2, col=1)

fig_sub.update_xaxes(type='date')
fig_sub.show()

In [507]:
### Can generate some summary statistics of automated recorded trips.
### Match trips by trip number in both actual and automated. Could be improved.

load_error_meter = []
dump_error_meter = []

for i in range(min(len(all_trips_for_machine), len(automated_trips))):
    temp_actual_load = all_trips_for_machine[i].load_latlon
    temp_actual_dump = all_trips_for_machine[i].dump_latlon
    temp_automated_load = automated_trips[i].load_latlon
    temp_automated_dump = automated_trips[i].dump_latlon
    
    load_error_meter.append(geopy.distance.geodesic(temp_actual_load, temp_automated_load).m)
    dump_error_meter.append(geopy.distance.geodesic(temp_actual_dump, temp_automated_dump).m)

In [508]:
dump_error_meter

[4.147445809989584,
 6927.4509993824795,
 194.7136168560992,
 8.558773762391462,
 156.25898784028647,
 2723.165057038564,
 6979.394881008932,
 52.267054843954,
 53.011288628940456,
 6933.2013128987555,
 4431.874506302255,
 81.85282173028608,
 6686.347725111872,
 6908.563571742692,
 6918.811068558061,
 29.83199916012405,
 6932.350472858376,
 294.50452918825823,
 433.48326313551905,
 47.595983643186045]